In [1]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

countt=0
fracture = []
nonfrac = []
for dirname, _, filenames in os.walk('/kaggle/input/fracatlas/FracAtlas/images/Fractured'):
    for filename in filenames:
        img_path  = os.path.join(dirname,filename)
        img = image.load_img(img_path, target_size=(224, 224,3))
        x_img = image.img_to_array(img)
        x_img = np.expand_dims(x_img, axis=0)
        x_imp = preprocess_input(x_img)
        fracture.append(x_img[0])

2024-03-16 10:39:21.138693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 10:39:21.138835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 10:39:21.305018: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
n = len(fracture)
n

717

In [3]:
countt = 0
for dirname, _, filenames in os.walk('/kaggle/input/fracatlas/FracAtlas/images/Non_fractured'):
    for filename in filenames:
        img_path  = os.path.join(dirname,filename)
        try:
            img = image.load_img(img_path, target_size=(224, 224,3))
            x_img = image.img_to_array(img)
            x_img = np.expand_dims(x_img, axis=0)
            x_imp = preprocess_input(x_img)
            if x_img is not None:
                nonfrac.append(x_img[0])
                countt+=1
                if countt==n:
                    break
        except:
            print(f"Error processing image {img_path}")

Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004347.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004297.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004148.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004177.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004291.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004290.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004288.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004263.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004029.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fractured/IMG0004189.jpg
Error processing image /kaggle/input/fracatlas/FracAtlas/images/Non_fr

In [45]:
fracture[0].shape

(224, 224, 3)

In [8]:
fracture_array = np.array(fracture)
nonfrac_array = np.array(nonfrac)

combined_array = np.concatenate((fracture_array, nonfrac_array), axis=0)

print(combined_array.shape)

(1434, 224, 224, 3)


In [9]:
sz1 = n
sz2 = n

array1 = np.array([1, 0])
array2 = np.array([0, 1])

repeated_array1 = np.tile(array1, (sz1, 1))
repeated_array2 = np.tile(array2, (sz2, 1))

result_array = np.concatenate((repeated_array1, repeated_array2), axis=0)

print(result_array.shape)
#print(result_array)

(1434, 2)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_array, result_array, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1147, 224, 224, 3)
X_test shape: (287, 224, 224, 3)
y_train shape: (1147, 2)
y_test shape: (287, 2)


In [11]:
# look how classification of images in dataset
fr = 0
nf = 0
for y in y_train:
    if y[0]==1:
        fr += 1
    elif y[0]==0:
        nf += 1
        
print(f"fractured images = {fr}\nnon fractured images = {nf}\ntotal images = {fr+nf}")

fractured images = 573
non fractured images = 574
total images = 1147


In [12]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

# Load the pre-trained VGG model without the top (fully connected) layers
base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))

# Create a new model with the VGG base model's input and output
vgg = Model(inputs=base_model.input, outputs=base_model.output)

# Load weights from the H5 file
vgg.load_weights('/kaggle/input/vgg16-weights-tf-keras/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [16]:
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

x = vgg.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=x)
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=categorical_crossentropy, metrics=['accuracy'])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,268,738 (512.19 MB)

 Trainable params: 134,268,738 (512.19 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
'''
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.keras", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=10,
    verbose="auto",
    callbacks=[checkpoint,early],
    validation_split=0.0,
    validation_data=(X_test, y_test),
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=100,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
'''

'\nfrom keras.callbacks import ModelCheckpoint, EarlyStopping\ncheckpoint = ModelCheckpoint("vgg16_1.keras", monitor=\'val_acc\', verbose=1, save_best_only=True, save_weights_only=False, mode=\'auto\')\nearly = EarlyStopping(monitor=\'val_acc\', min_delta=0, patience=20, verbose=1, mode=\'auto\')\nhist = model.fit(\n    x=X_train,\n    y=y_train,\n    batch_size=32,\n    epochs=10,\n    verbose="auto",\n    callbacks=[checkpoint,early],\n    validation_split=0.0,\n    validation_data=(X_test, y_test),\n    shuffle=True,\n    class_weight=None,\n    sample_weight=None,\n    initial_epoch=0,\n    steps_per_epoch=100,\n    validation_steps=None,\n    validation_batch_size=None,\n    validation_freq=1,\n)\n'

In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1219s 34s/step - accuracy: 0.5045 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1226s 34s/step - accuracy: 0.5044 - loss: 0.6931 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1274s 34s/step - accuracy: 0.5024 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 1227s 34s/step - accuracy: 0.4936 - loss: 0.6932 - val_accuracy: 0.4983 - val_loss: 0.6932
Epoch 5/10
 3/36 ━━━━━━━━━━━━━━━━━━━━ 17:26 32s/step - accuracy: 0.4757 - loss: 0.6933

KeyboardInterrupt: 

In [ ]:
'''
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

# Lets look at  fractured image
img_path = '/kaggle/input/fracatlas/FracAtlas/images/Fractured/IMG0000019.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x_img = image.img_to_array(img)
x_img = np.expand_dims(x_img, axis=0)
x_imp = preprocess_input(x_img)
features = model.predict(x_img)
print(features)

model.save('my_model.h5')
'''